<a href="https://colab.research.google.com/github/Patojukarthikeya73/Infosys_Springboard/blob/main/Final_work_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers gradio torch

import ast
import re
import gradio as gr
from transformers import pipeline

# ==================================================
# 🧠 BACKEND LOGIC — User Management, Models, Code Analysis
# ==================================================

# ---------------- User Management ----------------
users = {}

def register_user(username, password):
    if username in users:
        return False, "⚠️ Username already exists!"
    users[username] = {"password": password}
    return True, "✅ Registration successful! Please login."

def login_user(username, password):
    if username not in users:
        return False, "❌ User not found!"
    if users[username]["password"] != password:
        return False, "🚫 Incorrect password!"
    return True, "✅ Login successful!"

# ---------------- Model Initialization ----------------
phi_model = pipeline("text-generation", model="microsoft/phi-2")
codebert_model = pipeline("text-generation", model="microsoft/codebert-base")

# ---------------- Code Cleaning Helper ----------------
def extract_code_only(text):
    """Removes echoed prompt and retains only the actual code (for Python)."""
    match = re.search(r"(def |class )", text)
    if match:
        return text[match.start():].strip()
    return text.strip()

# ---------------- Python Code Analyzer ----------------
# ---------------- Python Code Analyzer ----------------
def explain_python_code(code):
    try:
        tree = ast.parse(code)
        explanation = []

        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                args = ", ".join(arg.arg for arg in node.args.args)
                explanation.append(f"Defines function '{node.name}' with args: {args}")
            elif isinstance(node, ast.For):
                explanation.append("Includes a for-loop.")
            elif isinstance(node, ast.While):
                explanation.append("Includes a while-loop.")
            elif isinstance(node, ast.If):
                explanation.append("Contains an if-else condition.")
            elif isinstance(node, ast.Assign):
                explanation.append("Performs variable assignment.")

        return " | ".join(set(explanation)) if explanation else "Simple code structure."
    except SyntaxError as e:
        return f"⚠️ Syntax Error in code: {e.msg} (Line {e.lineno})"
    except Exception as e:
        return f"⚠️ Could not analyze code: {str(e)}"


# ---------------- Code Generator ----------------
# ---------------- Code Generator ----------------
def generate_code(prompt, language):
    if language == "Python":
        raw_output = phi_model(prompt, max_new_tokens=180)[0]["generated_text"]

        # Remove echoed prompt
        code_candidate = raw_output.replace(prompt, "").strip()

        # Extract only first function or class definition
        match = re.search(r"(def .+?:|class .+?:)", code_candidate)
        if match:
            code = code_candidate[match.start():]
        else:
            code = code_candidate

        # Truncate after two line breaks if too long
        code = code.strip().split("\n\n")[0].strip()

        explanation = explain_python_code(code)
        return code, explanation

    elif language in ["JavaScript", "SQL"]:
        # Only return the first paragraph/block of code
        generated = codebert_model(prompt, max_new_tokens=180)[0]["generated_text"]
        cleaned = generated.replace(prompt, "").strip().split("\n\n")[0]
        return cleaned, f"Generated {language} code using CodeBERT. Review logic manually."

    return "⚠️ Unsupported language selected.", ""


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [2]:
def build_ui():
    with gr.Blocks(title="CodeGenie — Multi-Model Code Generator") as demo:
        gr.Markdown("# 🧠 CodeGenie — AI Code Generator with Python AST Analysis")

        # ---------- Login/Register Section ----------
        login_section = gr.Group(visible=True)
        with login_section:
            with gr.Tabs():
                with gr.Tab("Register"):
                    reg_user = gr.Textbox(label="Username")
                    reg_pass = gr.Textbox(label="Password", type="password")
                    reg_btn = gr.Button("Register")
                    reg_out = gr.Textbox(label="Status")
                    reg_btn.click(lambda u, p: register_user(u, p)[1], [reg_user, reg_pass], reg_out)

                with gr.Tab("Login"):
                    log_user = gr.Textbox(label="Username")
                    log_pass = gr.Textbox(label="Password", type="password")
                    log_btn = gr.Button("Login")
                    log_out = gr.Textbox(label="Status")

        # ---------- CodeGen Playground ----------
        codegen_section = gr.Group(visible=False)
        with codegen_section:
            gr.Markdown("## 💻 Code Playground")

            lang_select = gr.Radio(["Python", "JavaScript", "SQL"], label="Language", value="Python")
            prompt_input = gr.Textbox(label="🧩 Enter Problem Statement", placeholder="Describe your coding task...")
            gen_btn = gr.Button("🚀 Generate Code")
            code_output = gr.Textbox(label="🧩 Generated Code", lines=12)
            explain_output = gr.Textbox(label="🧠 Code Explanation", lines=6)

            # Button click → trigger backend code generation
            gen_btn.click(generate_code, [prompt_input, lang_select], [code_output, explain_output])

        # ---------- Login Handler ----------
        def handle_login(username, password):
            ok, msg = login_user(username, password)
            if ok:
                return gr.update(visible=False), gr.update(visible=True), msg
            return gr.update(visible=True), gr.update(visible=False), msg

        # Button click → trigger backend login
        log_btn.click(handle_login, [log_user, log_pass], [login_section, codegen_section, log_out])

    return demo

In [3]:
demo = build_ui()
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d21e6a89d7e9920a87.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
